In [1]:
import pandas as pd
import math

In [2]:
Training_data = pd.read_csv("mush_train.data", header = None)
Test_data = pd.read_csv("mush_test.data", header = None)

In [3]:
attr_c = len(Training_data.columns)-1
print(attr_c)

22


In [4]:
Training_data.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,e,f,f,n,t,n,f,c,b,n,...,s,p,g,p,w,o,p,n,y,d
1,e,x,y,y,t,a,f,c,b,n,...,y,w,w,p,w,o,p,n,y,p
2,p,x,y,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,g
3,p,x,s,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,p
4,e,x,f,w,t,l,f,w,n,w,...,s,w,w,p,w,o,p,u,v,d


In [5]:
attr_values = []

attr_values.append(['b', 'c', 'x', 'f', 'k', 's'])
attr_values.append(['f', 'g', 'y', 's'])
attr_values.append(['n','b','c','g','r','p','u','e','w','y'])
attr_values.append(['t','f'])
attr_values.append(['a','l','c','y','f','m','n','p','s'])
attr_values.append(['a','d','f','n'])
attr_values.append(['c','w','d'])
attr_values.append(['b','n'])
attr_values.append(['k','n','b','h','g','r','o','p','u','e','w','y'])
attr_values.append(['e','t'])
attr_values.append(['b','c','u','e','z','r','m'])
attr_values.append(['f','y','k','s'])
attr_values.append(['f','y','k','s'])
attr_values.append(['n','b','c','g','o','p','e','w','y'])
attr_values.append(['n','b','c','g','o','p','e','w','y'])
attr_values.append(['p','u'])
attr_values.append(['n','o','w','y'])
attr_values.append(['n','o','t'])
attr_values.append(['c','e','f','l','n','p','s','z'])
attr_values.append(['k','n','b','h','r','o','u','w','y'])
attr_values.append(['a','c','n','s','v','y'])
attr_values.append(['g','l','m','p','u','w','d'])

In [6]:
attr_data = {}
tree = {}

In [7]:
def initialise(tree):
    for i in range(attr_c):
        attr_data[i] = {}
        tree[i] = {}
        for j in attr_values[i]:
            tree[i][j] = -1

In [8]:
Training_data.loc[Training_data[1]=='f',[0,1]]

,0,1
0,e,f
10,p,f
13,e,f
15,p,f
17,e,f
25,p,f
29,p,f
30,e,f
38,p,f
39,p,f


In [9]:
def split(data,alist):
    rows = len(data)
    max_entropy = 1
    best_attr = -1
    for i in alist:
        entropy = 0
        for j in attr_values[i]:
            temp = data.loc[data[i+1]==j , [0,i+1]]
            temp_rows = len(temp)
            p = len(temp.loc[temp[0]=='p'])
            e = len(temp.loc[temp[0]=='e'])
            if temp_rows > 0:
                logp = math.log(p/temp_rows)/math.log(2) if p>0 else 0
                loge = math.log(e/temp_rows)/math.log(2) if e>0 else 0
                entropy += (-1)*(temp_rows/rows)*(((p/temp_rows)*logp)+((e/temp_rows)*loge))
        if entropy < max_entropy:
            max_entropy = entropy
            best_attr = i
    print(best_attr,max_entropy)
    return best_attr

In [10]:
def build_tree(data,root):
    max_height = 0
    nodes = 0
    attr_list = []
    for i in range(attr_c):
        attr_list.append(i)
    split_attr = split(data,attr_list)
    root = split_attr
    attr_list.remove(split_attr)
    attr_data[split_attr] = data
    list = []
    i = len(attr_values[split_attr])-1
    while i>=0:
        list.append([attr_values[split_attr][i],split_attr,0])
        i = i-1
    i = 0
    while(len(list)!=0):
        attr_val_pair = list.pop()
        nodes = nodes+1
        attr_to_spliton = attr_val_pair[1]
        val_to_spliton = attr_val_pair[0]
        complete_data = attr_data[attr_to_spliton]
        split_data = complete_data.loc[complete_data[attr_to_spliton + 1] == val_to_spliton]
        split_len = len(split_data)
        p = len(split_data.loc[split_data[0] == 'p'])
        e = len(split_data.loc[split_data[0] == 'e'])
        height = attr_val_pair[2]
        if height > max_height:
            max_height = height
        if(split_len == 0):
            p = len(complete_data.loc[complete_data[0] == 'p'])
            e = len(complete_data.loc[complete_data[0] == 'e'])
            if p>e:
                tree[attr_to_spliton][val_to_spliton] = 'p'
            else:
                tree[attr_to_spliton][val_to_spliton] = 'e'
        else:
            if p == split_len:
                tree[attr_to_spliton][val_to_spliton] = 'p'
            elif e == split_len:
                tree[attr_to_spliton][val_to_spliton] = 'e'
            else:
                best_attr = split(split_data,attr_list)
                attr_list.remove(best_attr)
                attr_data[best_attr] = split_data
                tree[attr_to_spliton][val_to_spliton] = best_attr
                i = len(attr_values[best_attr])-1
                while i>=0:
                    list.append([attr_values[best_attr][i],best_attr,height+1])
                    i = i-1
    for i in attr_list:
        del tree[i]
    return root,nodes,max_height+1 

In [11]:
root = None
initialise(tree)

In [12]:
root,node,height = build_tree(Training_data,root)

4 0.09140565768529109
19 0.07017946584096713
21 0.14877241668100136
2 0.0
7 0.0


In [13]:
print(tree)

{2: {'n': 'e', 'b': 'e', 'c': 'e', 'g': 'e', 'r': 'e', 'p': 'e', 'u': 'e', 'e': 'e', 'w': 'p', 'y': 'p'}, 4: {'a': 'e', 'l': 'e', 'c': 'p', 'y': 'p', 'f': 'p', 'm': 'p', 'n': 19, 'p': 'p', 's': 'p'}, 7: {'b': 'e', 'n': 'p'}, 19: {'k': 'e', 'n': 'e', 'b': 'e', 'h': 'e', 'r': 'p', 'o': 'e', 'u': 'e', 'w': 21, 'y': 'e'}, 21: {'g': 'e', 'l': 2, 'm': 'e', 'p': 'e', 'u': 'e', 'w': 'e', 'd': 7}}


In [14]:
print(node)

37


In [15]:
print(height)

4


In [16]:
def accuracy(dec_tree,data,root):
    accuracy = 0
    i = 0
    length = len(data)
    while i<length:
        result = root
        while True:
            result = dec_tree[result][data.iloc[i][result+1]]
            if result == 'p' or result == 'e':
                break
        if result == data.iloc[i][0]:
            accuracy = accuracy+1
        i = i+1
    return (accuracy/length)*100

In [17]:
training_accuracy = accuracy(tree,Training_data,root)
test_accuracy = accuracy(tree,Test_data,root)

In [18]:
training_accuracy

100.0

In [19]:
test_accuracy

100.0